# कुटुंब संबंधांचे ऑंटोलॉजी

हा उदाहरण [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners) चा एक भाग आहे, आणि [या ब्लॉग पोस्ट](https://habr.com/post/270857/) द्वारे प्रेरित आहे.

मला नेहमीच कुटुंबातील लोकांमधील वेगवेगळे संबंध लक्षात ठेवणे कठीण वाटते. या उदाहरणात, आपण कुटुंब संबंध परिभाषित करणारे ऑंटोलॉजी आणि प्रत्यक्ष वंशावळ घेऊ, आणि नंतर आपोआप निष्कर्ष काढून सर्व नातेवाईक शोधण्याचा मार्ग दाखवू.

### वंशावळ मिळवणे

उदाहरण म्हणून, आपण [Romanov Tsar Family](https://en.wikipedia.org/wiki/House_of_Romanov) ची वंशावळ घेऊ. कुटुंब संबंधांचे वर्णन करण्यासाठी सर्वात सामान्य स्वरूप [GEDCOM](https://en.wikipedia.org/wiki/GEDCOM) आहे. आपण Romanov कुटुंबाची वंशावळ GEDCOM स्वरूपात घेऊ:


In [1]:
!head -15 data/tsars.ged

0 HEAD
1 CHAR UTF8
1 GEDC
2 VERS 5.5
0 @0@ INDI
1 NAME Mihail Fedorovich /Romanov/
1 SEX M
1 BIRT
2 DATE 1613
1 DEAT 
2 DATE 1645
1 FAMS @41@
0 @1@ INDI
1 NAME Evdokija Lukjanovna /Streshneva/
1 SEX F


GEDCOM फाइल वापरण्यासाठी, आपण `python-gedcom` लायब्ररी वापरू शकतो:


In [2]:
import sys
!{sys.executable} -m pip install python-gedcom

ही लायब्ररी फाइल पार्सिंगसंबंधी काही तांत्रिक समस्या दूर करते, परंतु तरीही ती झाडातील सर्व व्यक्ती आणि कुटुंबांपर्यंत खूपच लो-लेव्हल प्रवेश देते. येथे आपण फाइल कशी पार्स करू शकतो आणि सर्व व्यक्तींची यादी कशी दाखवू शकतो ते दिले आहे:


In [3]:
from gedcom.parser import Parser
from gedcom.element.individual import IndividualElement
from gedcom.element.family import FamilyElement
g = Parser()
g.parse_file('data/tsars.ged')

In [4]:
d = g.get_element_dictionary()
[ (k,v.get_name()) for k,v in d.items() if isinstance(v,IndividualElement)]

[('@0@', ('Mihail Fedorovich', 'Romanov')),
 ('@1@', ('Evdokija Lukjanovna', 'Streshneva')),
 ('@2@', ('Aleksej Mihajlovich', 'Romanov')),
 ('@3@', ('Marija Ilinichna', 'Miloslavskaja')),
 ('@4@', ('Natalja Kirillovna', 'Naryshkina')),
 ('@5@', ('Marfa Matveevna', 'Apraksina')),
 ('@6@', ('Fedor Alekseevich', 'Romanov')),
 ('@7@', ('Sofja Aleksevna', 'Romanova')),
 ('@8@', ('Ivan V Alekseevich', 'Romanov')),
 ('@9@', ('Praskovja Fedorovna', 'Saltykova')),
 ('@10@', ('Ekaterina Ivanovna', 'Romanova')),
 ('@11@', ('Anna Ivanovna', 'Romanova')),
 ('@12@', ('Fridrih Vilgelm', 'Kurlandskij')),
 ('@13@', ('Karl Leopold', 'Meklenburg-Shverinskij')),
 ('@14@', ('Anna Leopoldovna', 'Meklenburg-Shverinskaja')),
 ('@15@', ('Anton Ulrih', 'Braunshvejg-Volfenbjuttelskij')),
 ('@16@', ('Ivan VI Antonovich', 'Braunshvejg-Volfenbjuttelskij')),
 ('@17@', ('Petr I Alekseevich', 'Romanov')),
 ('@18@', ('Evdokija Fedorovna', 'Lopuhina')),
 ('@19@', ('Ekaterina I Alekseevna', 'Mihajlova')),
 ('@20@', ('Ale

कुटुंबांबद्दल माहिती कशी मिळवायची ते येथे आहे. लक्षात घ्या की हे आपल्याला **ओळखपत्रांची** यादी देते, आणि अधिक स्पष्टता हवी असल्यास आपल्याला ती नावे मध्ये रूपांतरित करावी लागतील:


In [5]:
d = g.get_element_dictionary()
[ (k,[x.get_value() for x in v.get_child_elements()]) for k,v in d.items() if isinstance(v,FamilyElement)]

[('@41@', ['@0@', '@1@', '@2@']),
 ('@42@', ['@2@', '@3@', '@6@', '@7@', '@8@']),
 ('@43@', ['@8@', '@9@', '@10@', '@11@']),
 ('@44@', ['@13@', '@10@', '@14@']),
 ('@45@', ['@15@', '@14@', '@16@']),
 ('@46@', ['@2@', '@4@', '@17@']),
 ('@47@', ['@17@', '@18@', '@20@']),
 ('@48@', ['@20@', '@21@', '@22@']),
 ('@49@', ['@17@', '@19@', '@23@', '@24@']),
 ('@50@', ['@25@', '@23@', '@26@']),
 ('@51@', ['@26@', '@27@', '@28@']),
 ('@52@', ['@28@', '@30@', '@31@', '@33@']),
 ('@53@', ['@33@', '@34@', '@35@']),
 ('@54@', ['@35@', '@36@', '@37@']),
 ('@55@', ['@37@', '@38@', '@39@'])]

### कुटुंब ऑंटोलॉजी मिळवणे

आता, [कुटुंब ऑंटोलॉजी](https://raw.githubusercontent.com/blokhin/genealogical-trees/master/data/header.ttl) वर एक नजर टाकूया, जी सेमँटिक वेब ट्रिपलेट्सच्या संचाच्या स्वरूपात परिभाषित केली आहे. या ऑंटोलॉजीमध्ये `isUncleOf`, `isCousinOf` आणि इतर अनेक नातेसंबंध परिभाषित केले आहेत. हे सर्व नातेसंबंध मूलभूत प्रेडिकेट्स `isMotherOf`, `isFatherOf`, `isBrotherOf` आणि `isSisterOf` च्या आधारे परिभाषित केले आहेत. आपण ऑंटोलॉजीचा वापर करून स्वयंचलित तर्कशक्तीच्या मदतीने इतर सर्व नातेसंबंध शोधून काढणार आहोत.

`isAuntOf` प्रॉपर्टीच्या परिभाषेचे एक उदाहरण येथे दिले आहे, जी `isSisterOf` आणि `isParentOf` च्या संयोजनाने परिभाषित केली आहे (*आजी म्हणजे एखाद्याच्या पालकाची बहीण*).

```
fhkb:isAuntOf a owl:ObjectProperty ;
    rdfs:domain fhkb:Woman ;
    rdfs:range fhkb:Person ;
    owl:propertyChainAxiom ( fhkb:isSisterOf fhkb:isParentOf ) .
```


In [6]:
!head -20 data/onto.ttl

@prefix fhkb: <http://www.example.com/genealogy.owl#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.example.com/genealogy.owl#> a owl:Ontology .

fhkb:DomainEntity a owl:Class .

fhkb:Man a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;
                        owl:onProperty fhkb:hasSex ;
                        owl:someValuesFrom fhkb:Male ] ) ] .

fhkb:Woman a owl:Class ;
    owl:equivalentClass [ a owl:Class ;
            owl:intersectionOf ( fhkb:Person [ a owl:Restriction ;


### अनुमानासाठी ऑंटोलॉजी तयार करणे

सोप्या पद्धतीने, आपण एक ऑंटोलॉजी फाइल तयार करू ज्यामध्ये कौटुंबिक ऑंटोलॉजीमधील मूळ नियम आणि आमच्या GEDCOM फाइलमधील व्यक्तींच्या तथ्यांचा समावेश असेल. आपण GEDCOM फाइलमधून कुटुंबे आणि व्यक्तींबद्दलची माहिती काढू आणि ती त्रिकुटांमध्ये रूपांतरित करू.


In [7]:
!cp data/onto.ttl .

gedcom_dict = g.get_element_dictionary()
individuals, marriages = {}, {}

def term2id(el):
    return "i" + el.get_pointer().replace('@', '').lower()

out = open("onto.ttl","a")

for k, v in gedcom_dict.items():
    if isinstance(v,IndividualElement):
        children, siblings = set(), set()
        idx = term2id(v)

        title = v.get_name()[0] + " " + v.get_name()[1]
        title = title.replace('"', '').replace('[', '').replace(']', '').replace('(', '').replace(')', '').strip()

        own_families = g.get_families(v, 'FAMS')
        for fam in own_families:
            children |= set(term2id(i) for i in g.get_family_members(fam, "CHIL"))

        parent_families = g.get_families(v, 'FAMC')
        if len(parent_families):
            for member in g.get_family_members(parent_families[0], "CHIL"): # NB adoptive families i.e len(parent_families)>1 are not considered (TODO?)
                if member.get_pointer() == v.get_pointer():
                    continue
                siblings.add(term2id(member))

        if idx in individuals:
            children |= individuals[idx].get('children', set())
            siblings |= individuals[idx].get('siblings', set())
        individuals[idx] = {'sex': v.get_gender().lower(), 'children': children, 'siblings': siblings, 'title': title}

    elif isinstance(v,FamilyElement):
        wife, husb, children = None, None, set()
        children = set(term2id(i) for i in g.get_family_members(v, "CHIL"))

        try:
            wife = g.get_family_members(v, "WIFE")[0]
            wife = term2id(wife)
            if wife in individuals: individuals[wife]['children'] |= children
            else: individuals[wife] = {'children': children}
        except IndexError: pass
        try:
            husb = g.get_family_members(v, "HUSB")[0]
            husb = term2id(husb)
            if husb in individuals: individuals[husb]['children'] |= children
            else: individuals[husb] = {'children': children}
        except IndexError: pass

        if wife and husb: marriages[wife + husb] = (term2id(v), wife, husb)

for idx, val in individuals.items():
    added_terms = ''
    if val['sex'] == 'f':
        parent_predicate, sibl_predicate = "isMotherOf", "isSisterOf"
    else:
        parent_predicate, sibl_predicate = "isFatherOf", "isBrotherOf"
    if len(val['children']):
        added_terms += " ;\n    fhkb:" + parent_predicate + " " + ", ".join(["fhkb:" + i for i in val['children']])
    if len(val['siblings']):
        added_terms += " ;\n    fhkb:" + sibl_predicate + " " + ", ".join(["fhkb:" + i for i in val['siblings']])
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing%s ;\n    rdfs:label \"%s\" .\n" % (idx, added_terms, val['title']))

for k, v in marriages.items():
    out.write("fhkb:%s a owl:NamedIndividual, owl:Thing ;\n    fhkb:hasFemalePartner fhkb:%s ;\n    fhkb:hasMalePartner fhkb:%s .\n" % v)

out.write("[] a owl:AllDifferent ;\n    owl:distinctMembers (")
for idx in individuals.keys():
    out.write("    fhkb:" + idx)
for k, v in marriages.items():
    out.write("    fhkb:" + v[0])
out.write("    ) .")
out.close()

In [8]:
!tail onto.ttl

    fhkb:hasFemalePartner fhkb:i34 ;
    fhkb:hasMalePartner fhkb:i33 .
fhkb:i54 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i36 ;
    fhkb:hasMalePartner fhkb:i35 .
fhkb:i55 a owl:NamedIndividual, owl:Thing ;
    fhkb:hasFemalePartner fhkb:i38 ;
    fhkb:hasMalePartner fhkb:i37 .
[] a owl:AllDifferent ;
    owl:distinctMembers (    fhkb:i0    fhkb:i1    fhkb:i2    fhkb:i3    fhkb:i4    fhkb:i5    fhkb:i6    fhkb:i7    fhkb:i8    fhkb:i9    fhkb:i10    fhkb:i11    fhkb:i12    fhkb:i13    fhkb:i14    fhkb:i15    fhkb:i16    fhkb:i17    fhkb:i18    fhkb:i19    fhkb:i20    fhkb:i21    fhkb:i22    fhkb:i23    fhkb:i24    fhkb:i25    fhkb:i26    fhkb:i27    fhkb:i28    fhkb:i29    fhkb:i30    fhkb:i31    fhkb:i32    fhkb:i33    fhkb:i34    fhkb:i35    fhkb:i36    fhkb:i37    fhkb:i38    fhkb:i39    fhkb:i40    fhkb:i41    fhkb:i42    fhkb:i43    fhkb:i44    fhkb:i45    fhkb:i46    fhkb:i47    fhkb:i48    fhkb:i49    fhkb:i50    fhkb:i51    fhkb:i52    fhkb:i53    fhkb:

### अनुमान करणे

आता आपण या ऑंटोलॉजीचा उपयोग अनुमान आणि क्वेरीसाठी करू इच्छितो. यासाठी आपण [RDFLib](https://github.com/RDFLib) या लायब्ररीचा वापर करू, जी RDF ग्राफ वेगवेगळ्या स्वरूपात वाचणे, त्यावर क्वेरी करणे इत्यादीसाठी उपयुक्त आहे.

तार्किक अनुमानासाठी, आपण [OWL-RL](https://github.com/RDFLib/OWL-RL) लायब्ररीचा वापर करू, जी आपल्याला RDF ग्राफचे **क्लोजर** तयार करण्यास अनुमती देते, म्हणजेच सर्व शक्य संकल्पना आणि नातेसंबंध जे अनुमानित केले जाऊ शकतात ते जोडणे.


In [10]:
!{sys.executable} -m pip install rdflib
!{sys.executable} -m pip install git+https://github.com/RDFLib/OWL-RL.git

  Cloning https://github.com/RDFLib/OWL-RL.git to /tmp/pip-req-build-lbfzwi3m
  Running command git clone --filter=blob:none --quiet https://github.com/RDFLib/OWL-RL.git /tmp/pip-req-build-lbfzwi3m
  Resolved https://github.com/RDFLib/OWL-RL.git to commit a77e1791b88b54aace609bc6000aac14c7add4ff
  Preparing metadata (setup.py) ... done


चला ऑंटोलॉजी फाइल उघडू आणि त्यात किती त्रिकुटे आहेत ते पाहू:


In [11]:
import rdflib
from owlrl import DeductiveClosure, OWLRL_Extension

g = rdflib.Graph()
g.parse("onto.ttl", format="turtle")

print("Triplets found:%d" % len(g))

Triplets found:669


In [12]:
DeductiveClosure(OWLRL_Extension).expand(g)
print("Triplets after inference:%d" % len(g))

Triplets after inference:4246


### नातेवाईकांसाठी क्वेरी करणे

आता आपण ग्राफ क्वेरी करून लोकांमधील वेगवेगळ्या नातेसंबंधांचा शोध घेऊ शकतो. आपण **SPARQL** भाषा `query` पद्धतीसह वापरू शकतो. आपल्या उदाहरणात, आपल्या कुटुंबाच्या वंशवृक्षातील सर्व **काका** पाहूया:


In [13]:
qres = g.query(
    """SELECT DISTINCT ?aname ?bname
       WHERE {
          ?a fhkb:isUncleOf ?b .
          ?a rdfs:label ?aname .
          ?b rdfs:label ?bname .
       }""")

for row in qres:
    print("%s is uncle of %s" % row)

Fedor Alekseevich Romanov is uncle of Ekaterina Ivanovna Romanova
Aleksandr I Pavlovich Romanov is uncle of Aleksandr II Nikolaevich Romanov
Fedor Alekseevich Romanov is uncle of Anna Ivanovna Romanova


वेगवेगळ्या कुटुंबीय संबंधांशी प्रयोग करण्यासाठी मोकळ्या मनाने प्रयत्न करा. उदाहरणार्थ, तुम्ही `isAncestorOf` संबंध पाहू शकता, जो पुनरावृत्तीने दिलेल्या व्यक्तीचे सर्व पूर्वज परिभाषित करतो.

शेवटी, चला साफसफाई करूया!


In [14]:
!rm onto.ttl


---

**अस्वीकरण**:  
हा दस्तऐवज AI भाषांतर सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) वापरून भाषांतरित करण्यात आला आहे. आम्ही अचूकतेसाठी प्रयत्नशील असलो तरी, कृपया लक्षात घ्या की स्वयंचलित भाषांतरांमध्ये त्रुटी किंवा अचूकतेचा अभाव असू शकतो. मूळ भाषेतील मूळ दस्तऐवज हा अधिकृत स्रोत मानला जावा. महत्त्वाच्या माहितीसाठी, व्यावसायिक मानवी भाषांतराची शिफारस केली जाते. या भाषांतराचा वापर करून उद्भवणाऱ्या कोणत्याही गैरसमज किंवा चुकीच्या अर्थासाठी आम्ही जबाबदार राहणार नाही.
